In [ ]:
import sys
sys.path.append(r"../data/data_checking_code/")

import folder_tool
import vis_tool
import os
import csv
import cv2
import math
import time
import json
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology

from tqdm.notebook import tqdm as tqdm


In [ ]:
data_info, result_info_path = folder_tool.data_json_setting_load('./0_data_json_setting/0_run.json')
set_size = data_info['set_size']
# TODO 是否使用設定的切割方式，若沒有，隨機產生並輸出 split_df
name = data_info['name']
# Input
# TODO 預測的 mask 結果資料夾d:\plchao\Unet_Bl_binary\0_data_json_setting
pred_all_crown_list = glob.glob(f'./result/{name}/*_pred_raw/*.PNG')
pred_all_crown_list.sort()
assert len(pred_all_crown_list) != 0, 'wrong pred path'
# TODO 要比較的 label 資料夾
label_folder = data_info['y_train_dir']
# label_folder = '../data/7class_generate_tooth_mask/'
assert len(glob.glob(label_folder + '/*.PNG')) != 0, 'wrong label path'

In [ ]:
# Output
# TODO 輸出 csv 設定，會覆蓋原始檔案
output_csv_path = f"./IOU_csv/{name}.csv"

In [ ]:
def show_pixel_set(img_nparray):
    a = img_nparray
    unique, counts = np.unique(a, return_counts=True)
    return dict(zip(unique, counts))
def count_iou(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    intersection = np.logical_and(label_target, predict_target)
    union = np.logical_or(label_target, predict_target)
    iou_score = np.sum(intersection) / np.sum(union)
#     print('IOU val:',iou_score)
    return iou_score
def count_prdictPA(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    intersection = np.logical_and(label_target, predict_target)
    iou_score = np.sum(intersection) / np.sum(predict_target)
    return iou_score
def count_pixel_acc(label_img_gray, predict_img_gray, val):
    label_target = np.where(label_img_gray==val, 1, 0)
    predict_target = np.where(predict_img_gray==val, 1, 0)
    different = np.logical_xor(label_target, predict_target)
    diff_score = np.sum(different) / np.sum(np.ones_like(label_img_gray))
#     print('IOU val:',iou_score)
    return 1 - diff_score
def count_dist(label_img, pre_img):
    distance = []
    height, width = label_img.shape
    for col in range(width):
        label_piexl = np.where(label_img[:, col] == 255)[0]
        pre_piexl = np.where(pre_img[:, col] == 255)[0]
        if len(label_piexl) > 0 and len(pre_piexl) == 0:
            distance.append(np.nan)
        elif len(label_piexl) > 0:
            distance.append(abs(label_piexl[0] - pre_piexl[0]))
    return distance

def draw_contour(binary_img):
    sk_CEJ_mask = np.where(binary_img>=127, 1, 0)
    sk_CEJ_mask = morphology.skeletonize(sk_CEJ_mask)
    sk_CEJ_mask = np.where(sk_CEJ_mask==0, 0, 255)
    return sk_CEJ_mask


In [ ]:
record_iou = pd.DataFrame()
dist_combine_list = []
for pred_img_path in tqdm(pred_all_crown_list, total = len(pred_all_crown_list)):
    img_set_type = pred_img_path.split('\\')[-2]
    img_name = os.path.basename(pred_img_path)
    pre_img = cv2.imread(pred_img_path, 0)
    label_img = cv2.imread(label_folder + '/' + img_name, 0)
    if not os.path.exists(label_folder + '/' + img_name):
        print('no label image ', label_folder + '/' + img_name)
        continue
    label_img = np.where(label_img != 0, 255, 0)
    result_iou = count_iou(pre_img, label_img, 255)
    result_PA = count_pixel_acc(pre_img, label_img, 255)
    cont_label_img = cv2.imread(f'../3-Unet-mask-process/385_cej_1_pixel_512/CEJ_line_mask/{img_name}', 0)
    cont_pre_img = draw_contour(pre_img)
    img_id = img_name.replace('.PNG', '')
    cv2.imwrite(f'./z_debug/{img_id}_gt.PNG', cont_label_img)
    cv2.imwrite(f'./z_debug/{img_id}_pre.PNG', cont_pre_img)
    
    board_iou = count_iou(cont_label_img, cont_pre_img, 255)
    board_prePA = count_prdictPA(cont_label_img, cont_pre_img, 255)
    board_PA = count_pixel_acc(cont_label_img, cont_pre_img, 255)
    dist_list = count_dist(cont_label_img, cont_pre_img)
    dist_combine_list += dist_list

    record_iou = record_iou.append({'image': img_name, 'iou': result_iou, 'PA': result_PA, \
                                    'boarder_iou': board_iou, 'boarder_PA': board_prePA, \
                                    'dist_mean': np.mean(dist_list), 'image_set': img_set_type}\
        , ignore_index=True)
record_iou.to_csv(output_csv_path, index=False)

In [ ]:
def show_metrix(df):
    print(df['PA'].describe())
    print(df['iou'].describe())
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 2, 1)
    plt.hist(df['PA'], bins=100)
    plt.title('PA')
    plt.subplot(1, 2, 2)
    plt.hist(df['iou'], bins=100)
    plt.title('iou')
    plt.show()
def calculate_metrics(df, type=""):
    metrics = {
        'iou_mean': np.mean(df[type+'iou']),
        'PA_mean': np.mean(df[type+'PA']),
        'boarder_iou_mean': np.mean(df[type+'boarder_iou']),
        'boarder_precision_mean': np.mean(df[type+'boarder_PA']),
        'dist_mean': np.mean(df[type+'dist_mean']),
        'z_count': df[type+'iou'].notna().sum()
    }
    return metrics
result_df = pd.DataFrame()
for img_type_set in ['test', 'val', 'train']:
    # show_metrix(df)
    df = record_iou[record_iou.image_set == f'{img_type_set}_pred_raw']
    metrics = calculate_metrics(df)
    metrics['1_name'] = img_type_set
    result_df = result_df.append(metrics, ignore_index=True)
    if 'post_iou' in df.columns:
        metrics = calculate_metrics(df, 'post_')
        metrics['1_name'] = 'post_' + img_type_set
        result_df = result_df.append(metrics, ignore_index=True)
print(result_df.round(4))